# Day 7

## Part I

一个简单的DAG问题。首先创建一个类作为图的边Edge，不太会起名字，既然问题与旅行包相关，就叫BagEdge吧。外面的包叫做outer，里面的包叫做inner，边的权重值weight也就是一个大包可以放入小包的数量。为了方便显示，实现__repr__方法用于展示这个对象。

In [1]:
class BagEdge(object):
    def __init__(self, outer: str, inner: str, weight: int):
        self.outer = outer
        self.inner = inner
        self.weight = weight
    def __repr__(self) -> str:
        return f'{self.outer} --{self.weight}--> {self.inner}'

下面是输入数据的处理，要将输入数据变成合适的数据结构，包括上面定义的类。首先定义一个函数将类似`1 bright white`或`shiny gold`这样的输入数据转换为一个颜色和数量的元组：

In [2]:
from typing import Tuple

def bag_from_str(value: str) -> Tuple[str, int]:
    # 单独处理不包含其他任何包的情况
    if value == 'no other':
        return None, None
    parts = value.rstrip().split(' ')
    return (' '.join(parts[1:]), int(parts[0])) if len(parts) == 3 else (' '.join(parts), 1)

进行简单的单元测试：

In [3]:
assert(bag_from_str('1 bright white') == ('bright white', 1))
assert(bag_from_str('6 dotted black') == ('dotted black', 6))
assert(bag_from_str('shiny gold') == ('shiny gold', 1))

然后定义函数将输入的每一行转换成一个BagEdge的列表：

In [4]:
from typing import List
import re

def parse_line(line: str) -> List[BagEdge]:
    # 用正则表达式将一行里面所有的bag, bags, bags.都去掉
    line = re.sub(r' bags?[.]?', '', line.rstrip())
    parts = line.split(' contain ')
    outer, outer_weight = bag_from_str(parts[0])
    # 用循环添加每一对包的对应关系，也就是图中的边，因为逻辑稍微有点复杂，此处没有使用列表解析式
    bags = []
    for i in parts[1].split(', '):
        inner, inner_weight = bag_from_str(i)
        if inner:
            bags.append(BagEdge(outer, inner, inner_weight//outer_weight))
    return bags

下面做几个验证，没有使用断言，肉眼可观察是否正确：

In [5]:
parse_line('light red bags contain 1 bright white bag, 2 muted yellow bags.')

[light red --1--> bright white, light red --2--> muted yellow]

In [6]:
parse_line('vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.')

[vibrant plum --5--> faded blue, vibrant plum --6--> dotted black]

In [7]:
parse_line('dotted black bags contain no other bags.')

[]

解析输入数据的帮助函数都定义好了，下面来读取文件，解析每一行，形成一个边的列表：

In [8]:
def read_input(input_file: str) -> List[BagEdge]:
    result = []
    with open(input_file) as fn:
        line = fn.readline()
        while line:
            result += parse_line(line)
            line = fn.readline()
    return result

用测试用例1来看看读取和解析的效果，一切看着挺美好：

In [9]:
testcase = read_input('testcase1.txt')
testcase

[light red --1--> bright white,
 light red --2--> muted yellow,
 dark orange --3--> bright white,
 dark orange --4--> muted yellow,
 bright white --1--> shiny gold,
 muted yellow --2--> shiny gold,
 muted yellow --9--> faded blue,
 shiny gold --1--> dark olive,
 shiny gold --2--> vibrant plum,
 dark olive --3--> faded blue,
 dark olive --4--> dotted black,
 vibrant plum --5--> faded blue,
 vibrant plum --6--> dotted black]

第一部分问题是逆向搜索图，我们将边的列表转换成一个字典方便处理，然后使用深度优先搜索，将整个图中能包含shiny gold的颜色都添加都一个集合set中，最后这个集合中的元素个数即为答案：

In [10]:
from typing import Dict, Set

def search_outer_recursive(inners: Dict[str, List[BagEdge]], inner_bag: str, result: Set[str]):
    for bag in inners.get(inner_bag, []):
        result.add(bag.outer)
        search_outer_recursive(inners, bag.outer, result)

def part1_solution(bags: List[BagEdge]) -> int:
    inners = {}
    for bag in bags:
        inners.setdefault(bag.inner, [])
        inners[bag.inner].append(bag)
    result = set()
    search_outer_recursive(inners, 'shiny gold', result)
    return len(result)

试一试测试用例：

In [12]:
assert(part1_solution(testcase) == 4)

没毛病，可以看看第一部分的结果了：

In [13]:
bags = read_input('input.txt')
part1_solution(bags)

246

## Part II

第二部分是正向搜索图，并且需要使用到边的权重进行计算。同样为了方便起见，将边的列表转换成正向的字典方便遍历，还是采用深度优先方式计算每一条路线的数量总和，最后累加即为最终结果：

In [14]:
def count_inner_recursive(outers: Dict[str, List[BagEdge]], outer_bag: str, init_qty: int) -> int:
    result = 0
    for bag in outers.get(outer_bag, []):
        inner_bag_count = init_qty * bag.weight
        result += inner_bag_count
        result += count_inner_recursive(outers, bag.inner, inner_bag_count)
    return result

def part2_solution(bags: List[BagEdge]) -> int:
    outers = {}
    for bag in bags:
        outers.setdefault(bag.outer, [])
        outers[bag.outer].append(bag)
    return count_inner_recursive(outers, 'shiny gold', 1)

两个测试用例的结果：

In [15]:
assert(part2_solution(testcase) == 32)

testcase2 = read_input('testcase2.txt')
assert(part2_solution(testcase2) == 126)

最后是第二部分的结果：

In [16]:
part2_solution(bags)

2976